<a href="https://colab.research.google.com/github/Vedalaxman/Audio_to_Text/blob/main/Wishper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import torch

import warnings
warnings.filterwarnings('ignore')

In [31]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [10]:
import transformers

In [11]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, BitsAndBytesConfig
model_id = "Na0s/Medical-Whisper-Large-v3"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading model on device: {device}")
processor = AutoProcessor.from_pretrained(model_id)
print("Processor Loaded")
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.float16
    )
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        use_safetensors=True
    )
    print("Model Loaded in 8-bit precision with float16 compute dtype.")
else:
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id,
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True,
        use_safetensors=True
    ).to(device)
    print("Model Loaded on CPU in float32 precision.")

print("Model Loaded")

Loading model on device: cuda
Processor Loaded


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model Loaded in 8-bit precision with float16 compute dtype.
Model Loaded


In [4]:
file_path = '/content/eval-00000-of-00001.parquet'

try:
    df = pd.read_parquet(file_path)
    print('Done')
except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"Error reading Parquet: {e}")

Done


In [5]:
df

,audio,text,duration
0,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,then our patients diagnosed. Those things are ...,11.0
1,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,a simple way is to validate the Institute of M...,10.0
2,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,They don't know what to look for in sleep stud...,13.0
3,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,"But it wouldn't take very long, I think, from ...",11.0
4,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,"Vicky Whittemore: Thank you, Cindy. Lots of pe...",12.0
...,...,...,...
2811,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,SUCH IMPORTANT ADVICE SUCH AN IMPORTANT TOPIC....,9.0
2812,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,"IS IN CRISIS, PLEASE CALL 988, THE SUICIDE AND...",8.0
2813,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,MENTAL HEALTH CAN BE AN UNCOMFORTABLE TOPIC TO...,6.0
2814,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...,BUT THE TEENS I SPOKE WITH SAID THAT DISCUSSIO...,7.0


In [6]:
from datasets import Dataset, Audio
import librosa

In [7]:
/*dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

columns_to_remove = []
if 'audio_id' in dataset.features:
    columns_to_remove.append('audio_id')
if 'duration' in dataset.features:
    columns_to_remove.append('duration')

if columns_to_remove:
    dataset = dataset.remove_columns(columns_to_remove)

In [8]:
dataset

Dataset({
    features: ['audio', 'text'],
    num_rows: 2816
})

In [13]:
test_df = dataset.select(range(5))

In [10]:
test_df

Dataset({
    features: ['audio', 'text'],
    num_rows: 2
})

In [11]:
def transcribe_audio_batch(batch):
    audio_inputs = [item["array"] for item in batch["audio"]]
    input_features = processor.feature_extractor(
        audio_inputs,
        sampling_rate=16000,
        return_tensors="pt"
    ).input_features.to(device)
    if device == "cuda":
        input_features = input_features.to(torch.float16)
    predicted_ids = model.generate(input_features, language="en", task="transcribe")
    transcriptions = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
    return {"transcribed_text": transcriptions}

In [ ]:
transcription_results_dataset = test_df.map(
    transcribe_audio_batch,
    batched=True,
    batch_size=1,
    remove_columns=["audio"]
)

df_final = transcription_results_dataset.to_pandas

In [ ]:
df_final

In [12]:
import soundfile as sf
import librosa

In [13]:
def transcribe_single_audio(audio_path):
    speech_array, sr = librosa.load(audio_path, sr=16000)
    input_features = processor.feature_extractor(
        speech_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features.to(device)
    if device == "cuda":
        input_features = input_features.to(torch.float16)
    predicted_ids = model.generate(input_features, language="en", task="transcribe")
    transcription = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    return transcription

In [14]:
audio_path = '/content/OSR_us_000_0011_8k.wav'

In [15]:
text = transcribe_single_audio(audio_path)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [16]:
text

" The boy was there when the sun rose. A rod is used to catch pink salmon. The source of the huge river is the clear spring. Kick the ball straight and follow through. Help the woman get back to her feet. A pot of tea helps to pass the evening. Smokey fires lack flame and heat. The soft cushion broke the man's fall. The salt breeze came across the sea. The girl at the "